# NFe Prompt Parser

# Import Libraries

In [88]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os
import json
import time

## Load Data

In [89]:
df = pd.read_csv("../data/nfe_produto_1000_estruturado_anotado_test.csv")
df.head()

,text
0,"ARROZ PARBOLIZADO,LONGO FINO T1,PCT 1.0KG,BLUE..."
1,BISCOITO MAISENA C/ 400G - MABEL
2,FILé DE MERLUZA CONG.
3,AçúCAR CRISTAL PINDORAMA FD30X1KG
4,MINGAU DE MILHO VERDE COM CòCO


## NFe Parser Function

In [90]:
def nfe_parser_gpt(df, openai_model="gpt-4"):
    '''Function to parse NFE descriptions into structured JSON format using GPT-4'''

    # Load OpenAI API key from .env file
    _ = load_dotenv(find_dotenv())
    openai_api_key = os.environ["OPENAI_API_KEY"]

    client = OpenAI(api_key=openai_api_key)
    
    response_list = []
    for item in df['text']:
        response = client.chat.completions.create(
            model=openai_model,
            messages=[
                {
                "role": "system",
                "content": "Você é um assistente que organiza itens de notas fiscais. "
                },
                {
                "role": "user",
                "content": "Para cada descrição oferecida, organize os dados em formato JSON, nos campos \"produto\", \"marca\", \"quantidade\", \"unidade\", \"apresentacao\" e \"observacao\".\nComplete o nome do produto se estiver abreviado, utilizando a primeira letra maiúscula seguida de minúsculas. Remova quaisquer adjetivos ou descrições adicionais que não façam parte do nome do produto.\nA marca deve ser preenchida apenas quando uma segunda entidade for mencionada de forma que possa ser interpretada como tal; caso contrário, deixe o campo \"marca\" vazio (\"\").\nA \"quantidade\" deve refletir o peso total ou o conteúdo total de um único item ou pacote.\nA \"unidade\" deve estar na unidade do SI correspondente (g, kg, etc.), quando aplicável.\nSe a descrição mencionar um número de caixas ou pacotes, detalhe isso no campo \"apresentacao\" de forma apropriada (ex: \"pacote de 200g\", \"garrafa de 2l\").\nCorrija abreviações de unidades de medida para suas formas corretas no SI (\"GR\", \"GRS\" para \"g\"; \"KGS\", \"KLS\" para \"kg\").\nConverta números com vírgula para decimais em formato americano (com ponto).\nSe um campo não puder ser preenchido com as informações fornecidas, use aspas duplas (\"\") para indicar a ausência de dados.\nO campo \"observacao\" deve ser usado para qualquer informação adicional que não se encaixe nos outros campos.\nNão adicione campos extras ao JSON. Qualquer informação adicional deve ser incluída no campo \"observacao\".\n \n### Descrição:\nARROZ POLIDO AGULHINHA TIPO 01 REI DA PANELA 5KG\n \n### Resposta:\n{row[\"estruturado\"]}\n \n### Fim"
                },
                {
                "role": "assistant",
                "content": item
                }
            ],
            temperature=0,
            max_tokens=600,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        response_list.append(response.choices[0].message.content)
        time.sleep(1)
    return response_list

## Test of NFe Parser with a little dataframe

In [66]:
df_littler = df.head(3)
openai_model = "gpt-4"
#openai_model = "gpt-3.5-turbo-1106"
response_list = nfe_parser_gpt(df_littler, openai_model)

print("Lenght of response_list: ",len(response_list))

for item in response_list:
    print(item)
    print('------------------------')

Lenght of response_list:  3
{
"produto": "Arroz parbolizado",
"marca": "Blue Vile",
"quantidade": "1.0",
"unidade": "kg",
"apresentacao": "Pacote 1.0kg",
"observacao": "Longo fino tipo 1"
}
------------------------
{
"produto": "Biscoito Maizena",
"marca": "Mabel",
"quantidade": 400,
"unidade": "g",
"apresentacao": "Pacote de 400g",
"observacao": ""
}
------------------------
{ "produto": "Filé de Merluza", "marca": "", "quantidade": "", "unidade": "", "apresentacao": "Congelado", "observacao": "" }
------------------------


## Use NFe Parser
### RUN ONLY IF YOU ARE SURE

In [91]:
# To select the model, change the variable openai_model
openai_model = "gpt-4"
# openai_model = "gpt-3.5-turbo-1106"

response_list = nfe_parser_gpt(df, openai_model)

print("Lenght of response_list: ",len(response_list))
print(response_list[0])

Lenght of response_list:  100
{
"produto": "Arroz parbolizado",
"marca": "Blue Vile",
"quantidade": "1.0",
"unidade": "kg",
"apresentacao": "Pacote 1.0kg",
"observacao": "Longo fino tipo 1"
}


## JSON dump

In [92]:
# Dump the response_list into a JSON file
with open(f'../data/nfe_produto_1000_estruturado_anotado_test_{openai_model}.json', 'w') as f:
    json.dump(response_list, f)

## Load JSON file dumped with error correct

In [103]:
# Load the JSON file
#openai_model = "gpt-3.5-turbo-1106"
with open(f'../data/nfe_produto_1000_estruturado_anotado_test_{openai_model}.json') as f:
    response_list = json.load(f)

## Convert JSON Response to Python dictionary

In [104]:
# Convert JSON strings from response_list to Python dictionaries
# list_of_dicts = [json.loads(d) for d in response_list]
# list_of_dicts

list_of_dicts = []

for d in response_list:
    try:
        dict_item = json.loads(d)
        list_of_dicts.append(dict_item)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e} in string: {d}")
        # Handle the error or skip the item


## Create Response Datafreme 

In [105]:
# Create a pandas DataFrame from list_of_dicts
df_response = pd.DataFrame(list_of_dicts)
df_response['quantidade'] = df_response['quantidade'].astype(str)
df_response

,produto,marca,quantidade,unidade,apresentacao,observacao
0,Arroz Parbolizado,Blue Vile,1.0,kg,pacote de 1.0kg,Longo Fino T1
1,Biscoito Maisena,Mabel,400,g,,
2,Filé de Merluza,,1,unidade,pacote,Congelado
3,Açúcar Cristal,Pindorama,30,kg,pacote de 1kg,
4,Mingau de Milho Verde,,1,unidade,,com côco
...,...,...,...,...,...,...
95,Frutti Life Uva,,1,l,,Sem açúcar
96,Colorífico,São Marcos,100,g,,
97,Queijo ralado,Zaeli,50,g,,
98,Fermento,Fleischmann,450,g,,PF


## Save DataFrame to Parquet File

In [106]:
# Save DataFrame to parquet file
parquet_file_name = f'../data/nfe_produto_1000_estruturado_anotado_test_{openai_model}.parquet'
df_response.to_parquet(parquet_file_name)

## Teste Parquet File in a new DataFrame

In [107]:
# Testando a leitura do arquivo parquet
df_from_parquet = pd.read_parquet(parquet_file_name)
df_from_parquet.head()

,produto,marca,quantidade,unidade,apresentacao,observacao
0,Arroz Parbolizado,Blue Vile,1.0,kg,pacote de 1.0kg,Longo Fino T1
1,Biscoito Maisena,Mabel,400,g,,
2,Filé de Merluza,,1,unidade,pacote,Congelado
3,Açúcar Cristal,Pindorama,30,kg,pacote de 1kg,
4,Mingau de Milho Verde,,1,unidade,,com côco


In [108]:
# Test length of parquet file
df_gpt4 = pd.read_parquet('../data/nfe_produto_1000_estruturado_anotado_test_gpt-4.parquet')
gpt_3_5 = pd.read_parquet('../data/nfe_produto_1000_estruturado_anotado_test_gpt-3.5-turbo-1106.parquet')
print(len(df_gpt4))
print(len(gpt_3_5))

98
100
